# Tarefa 2: Usar o SageMaker Debugger

Neste laboratório, você vai usar o Amazon SageMaker Debugger para analisar, detectar e receber alertas sobre gargalos, taxas de utilização de recursos e vários problemas de treinamento durante os trabalhos de treinamento.

## Tarefa 2.1: Configuração do ambiente

Instale os pacotes e as dependências.

In [ ]:
%%capture

#install a compiler
%conda install -c conda-forge gcc -y

**Atenção:** o GCC pode exigir até cinco minutos para ser instalado.

In [ ]:
#install-dependencies

%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install -U sagemaker
%pip install -U smdebug
%pip install -U shap

In [ ]:
#import libraries and set variable values

import sys
import boto3
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sagemaker
import sagemaker_datawrangler 
import shap

from mpl_toolkits.axes_grid1 import host_subplot

from sagemaker.debugger import (
    CollectionConfig,
    DebuggerHookConfig,
    FrameworkProfile,
    ProfilerConfig,
    ProfilerRule,
    Rule,
    rule_configs,
    TensorBoardOutputConfig
)

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sagemaker.xgboost.estimator import XGBoost
from smdebug.core import modes
from smdebug.trials import create_trial

base_job_name = "lab-7-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "lab-7-smdebugger"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
save_interval = 5

Depois, importe os conjuntos de dados.

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

Divida o conjunto de dados da seguinte maneira: treinamento (70%), validação (20%) e teste (10%). Os conjuntos de dados de treinamento e validação são usados para criar o modelo neste laboratório. Você não vai usar o conjunto de teste neste laboratório.

In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

feature_names = list(train_data.columns)

Agora, carregue o conjunto de dados no Amazon Simple Storage Service (Amazon S3).

In [ ]:
#upload-dataset
sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

## Tarefa 2.2: Modificar o script de treinamento para ativar o SageMaker Debugger

Você precisa modificar o script de treinamento usado no laboratório anterior para salvar os tensores em um bucket do S3 de saída específico, determinar quais tensores serão salvos e registrar os ganchos de depuração.

Para treinar o modelo, será necessário configurar o seguinte:
- **Debugger Hook Parameters** (Parâmetros de gancho do Debugger) para ajustar os intervalos de salvamento dos tensores de saída nas fases de treinamento
- **Debugger Rule Object** (Objeto de regra do Debugger) para salvar os tensores de saída para avaliação

Em **Debugger Hook Parameters** (Parâmetros de gancho do Debugger), configure as coleções de tensores integradas **metrics**, **feature_importance**, **full_shap** e **average_shap** a serem capturadas durante o treinamento. Elas são configuradas em **collection_configs** de **debugger_hook_config**. As coleções de tensores integradas **full_shap** e **average_shap** usam valores Shapley (SHAP). O SHAP explica uma previsão de machine learning (ML) considerando que cada valor de recurso de uma instância de dados de treinamento é um agente em um jogo em que a previsão é o ganho. Ele também indica como distribuir o ganho de maneira justa entre os recursos. Consulte [Linhas de base do SHAP para explicabilidade](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html) para saber mais sobre o SHAP.

Em **Debugger Rule Object** (Objeto de regra do Debugger), configure as seguintes **rule_configs** em **rules**:
- [Relatório de criação de perfil](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report.html#debugger-profiling-report): executa regras para detecção de gargalos do sistema e gera automaticamente um relatório de definição de perfil.
- [Relatório de treinamento do XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report-xgboost.html): executa um relatório abrangente do XGBoost.
- [Sobreajuste](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overfit): detecta se o modelo está sendo sobreajustado para os dados de treinamento comparando a validação e as perdas de treinamento.
- [Excesso de treinamento](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overtraining): detecta se um modelo está com excesso de treinamento. Após várias iterações de treinamento em um modelo com comportamento bom (redução de perda de treinamento e validação), o modelo se aproxima do mínimo da função de perda e não apresenta mais melhoria. Se o treinamento continuar no modelo, a perda de validação poderá começar a aumentar porque o modelo começará a ficar com sobreajuste.
- [A perda não está diminuindo](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#loss-not-decreasing): detecta quando a perda não está diminuindo em valor a uma taxa adequada. Essas perdas precisam ser escalares. 

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type='ml.m5.4xlarge',
    #Set the hyperparameters
    hyperparameters= {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "binary:logistic",
        "num_round": "300",
    },
    sagemaker_session=sagemaker_session,
    max_run=1800,
    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[  # For each of these, a new processing job will be run later in the lab
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
    ),
    #Set the Debugger Profiler Configuration
    profiler_config = ProfilerConfig(
        system_monitor_interval_millis=500,
        framework_profile_params=FrameworkProfile()

    ),
    #Configure the Debugger Rule Object
    rules = [
        ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
        Rule.sagemaker(rule_configs.create_xgboost_report()),  
        Rule.sagemaker(rule_configs.overfit()),
        Rule.sagemaker(rule_configs.overtraining()),
        Rule.sagemaker(rule_configs.loss_not_decreasing(),
            rule_parameters={
                "collection_names": "metrics",
                "num_steps": str(save_interval * 2),
            }
        )
    ]
)

## Tarefa 2.3: Executar o trabalho de treinamento ativado pelo Debugger

Agora, treine o modelo do XGBoost usando o script ativado pelo Debugger. O treinamento exige cerca de cinco a dez minutos para ser executado. Você pode continuar na próxima tarefa enquanto o trabalho de treinamento está em execução e monitorar o andamento do trabalho usando o SageMaker Debugger.

In [ ]:
#train-model
xgb.fit(
    inputs = data_inputs
)

## Tarefa 2.4: Monitorar o status do trabalho de treinamento

No SageMaker Studio, você pode examinar os componentes de teste, incluindo todos os trabalhos do SageMaker Debugger que iniciou. Neste laboratório, você criou os trabalhos: **Relatório do XGBoost**, **Sobreajuste**, **Excesso de treinamento** e **A perda não está diminuindo**. Explore-os no SageMaker Studio.

A próxima etapa abrirá uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **lab_7.ipynb** para a lateral ou escolha (clique com o botão direito) a aba **lab_7.ipynb** e selecione **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora o grupo de recursos.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções.

1. Selecione o ícone **Página inicial do SageMaker**.
2. Escolha **Experiments** (Experimentos).

O SageMaker Studio abre a aba **Experiments** (Experimentos).

3. Escolha **Unassigned runs** (Execuções não atribuídas).
4. Na lista, selecione o **Name** (Nome) do trabalho, que tem o Tipo **SageMakerTrainingJob**.

São exibidos detalhes do trabalho de treinamento.

5. No lado esquerdo, selecione a aba **Debugger**.

O SageMaker Debugger mostra o status do trabalho de treinamento, que você pode monitorar enquanto o treinamento do modelo está em execução. Após a conclusão, você verá o status dos problemas de treinamento especificados.

A análise é concluída quando as linhas **Status** (Status) exibem **No Issues Found** (Nenhum problema encontrado) ou **Issues Found** (Problemas encontrados). As regras do Debugger podem exigir até nove minutos para serem concluídas.

Se forem encontrados problemas no modelo, talvez você queira corrigi-los. Foram encontrados problemas nos trabalhos? 

Neste laboratório, você não vai resolver os problemas encontrados. No entanto, se você quiser resolver os problemas encontrados, poderá aplicar uma combinação de processamento de conjunto de dados e novo treinamento do modelo com hiperparâmetros ajustados.

6. Quando a análise for concluída, retorne à aba do notebook chamada **lab_7.ipynb**.

## Tarefa 2.5: Executar a análise pós-treinamento

Com o SageMaker Debugger, você pode criar logs de trabalho de processamento no Amazon CloudWatch que poderá usar para configurar alarmes personalizados. Aqui, você imprime o local em que os logs são armazenados para cada métrica avaliada.

In [ ]:
#print-urls
def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
    """Helper function to get the rule job name with correct casing"""
    return "{}-{}-{}".format(
        training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
    )


def _get_cw_url_for_rule_job(rule_job_name, region):
    return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(
        region, region, rule_job_name
    )


def get_rule_jobs_cw_urls(xgb):
    region = boto3.Session().region_name
    training_job = xgb.latest_training_job
    training_job_name = training_job.describe()["TrainingJobName"]
    rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]

    result = {}
    for status in rule_eval_statuses:
        if status.get("RuleEvaluationJobArn", None) is not None:
            rule_job_name = _get_rule_job_name(
                training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"]
            )
            result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(
                rule_job_name, region
            )
    return result


get_rule_jobs_cw_urls(xgb)

Os tensores podem ser recuperados por coleções-padrão como pesos, gradientes, vieses e perdas que o SageMaker Debugger cria por meio do trabalho de treinamento, além das coleções personalizadas dos tensores. Gere uma lista de nomes e valores para os tensores salvos a fim de determinar quais tensores devem ser colocados em gráfico para análise adicional.

In [ ]:
#retrieve-names
trial = create_trial(xgb.latest_job_debugger_artifacts_path())
trial.tensor_names()

In [ ]:
#retrieve-values
trial.tensor("average_shap/f1").values()

In [ ]:
#plot-tensors
shap_values = trial.tensor("full_shap/f10").value(trial.last_complete_step)
shap_no_base = shap_values[:, :-1]
shap_base_value = shap_values[0, -1]
shap.summary_plot(shap_no_base, plot_type="bar", feature_names=feature_names)

## Tarefa 2.6: Acessar o painel de informações do SageMaker Debugger

1. Retorne à aba **Experiments** (Experimentos).
2. À esquerda, selecione **Debugger**.
3. Na seção **Debugger insights** (Informações do depurador), selecione o **lab-7-smdebugger-job** disponível na lista **Training job name** (Nome do trabalho de treinamento).

O SageMaker Studio abre uma nova aba **Debugger insights tab** (Informações do Debugger) para o trabalho e inicia o carregamento de dados.

O SageMaker Debugger fornece uma visão geral do desempenho do treinamento do modelo nas instâncias do Amazon Elastic Compute Cloud (Amazon EC2). Explore o SageMaker Debugger no SageMaker Studio e examine os detalhes contidos nos relatórios.

A aba **Systems Metrics** (Métricas de sistemas) inclui as seguintes seções:

- **Resource utilization summary** (Resumo de utilização de recurso)
- **CPU utilization summary** (Resumo de utilização de CPU)
- **GPU Utilization summary** (Resumo de utilização de GPU)

A aba **Rules** (Regras) inclui as seguintes **Insights** (Informações): 

- **BatchSize**
- **LowGPUUtiliztion**
- **CPUBottleneck**
- **GPUMemoryIncrease**
- **StepOutlier**
- **MaxInitializationTime**
- **IOBottleneck**
- **LoadBalancing**

Os dados serão preenchidos nos gráficos e nas tabelas se algum problema for encontrado.

4. Você pode baixar um relatório do Debugger clicando em <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Download report (Baixar relatório)</span> próximo à parte superior da aba **Debugger insights** (Informações do Debugger).

### Conclusão

Parabéns! Você usou o SageMaker Debugger para analisar, detectar e criar alertas para possíveis problemas no treinamento do modelo. Agora você pode usar as informações geradas pelos relatórios e alertas para treinar e criar de maneira eficiente um modelo mais eficaz. O próximo laboratório se concentra no uso do SageMaker Clarify para detectar viés e fornecer explicabilidade para previsões de modelo.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.